In [1]:
#%pip install gradio

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\USER\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
#실습1
import gradio as gr
 
def greet(name):
    return "Hello!!!" + name + "World!!!"
 
with gr.Interface(fn=greet, inputs="text", outputs="text") as demo:
    pass
 
demo.launch()

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [8]:
#실습2
import gradio as gr
 
def greet(name):
    return "Hello!!!" + name + "World!!!"
 
with gr.Blocks() as demo:
    name = gr.Textbox(label='이름')
    output = gr.Textbox(label='출력')
    submit_button = gr.Button('전송')
    submit_button.click(fn=greet, inputs=[name], outputs=[output], api_name='전송')
 
demo.launch()

Running on local URL:  http://127.0.0.1:7861

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


In [12]:
#실습3 이벤트 리스너
def welcome(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Hello World!
    Start typing below to see the output
    """)
    inp = gr.Textbox(placeholder='What is your name?')
    out = gr.Textbox()
    inp.change(welcome,inp,out)

demo.launch()


Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [13]:
import requests
import gradio as gr

AZURE_OPENAI_ENDPOINT = "https://b22-openai.openai.azure.com/openai/deployments/b22-gpt4o/chat/completions?api-version=2024-02-15-preview"
AZURE_OPENAI_API_KEY = "1dbcc70ac05e48b68207e09ed5667d74"
DEPLOYMENT_NAME = "b22-gpt4o" 

def chatgpt_response(prompt, history):
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
  
    payload = {
        "messages": [
            {
                "role": "system",
                "content": "You are an AI assistant that helps people find information."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }
  
    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}",
        headers=headers,
        json=payload
    )
  
    result = response.json()
    print(result)  # API 응답을 출력하여 확인

    if 'choices' not in result:
        return "Error: API response does not contain 'choices' key", history

    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))
    return '', history

def greet(name):
    name_length = len(name)
    if name_length > 5:
        return "Hello!!! " + name + " World!!!"
    else:
        return ""

with gr.Blocks() as demo:
    gr.Markdown("#  ChatGPT World!!!")
    chatbot = gr.Chatbot()
    
    with gr.Column():
        with gr.Row():
            prompt = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요", scale=3)
            send_button = gr.Button(">", scale=1)
            clear_button = gr.Button("X")
        send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
        prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
        clear_button.click(lambda: None, None, queue=False)
    demo.launch()


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [32]:
#이전대화고려
import requests
import gradio as gr

AZURE_OPENAI_ENDPOINT = "https://b22-openai.openai.azure.com/openai/deployments/b22-gpt4o/chat/completions?api-version=2024-02-15-preview"
AZURE_OPENAI_API_KEY = "1dbcc70ac05e48b68207e09ed5667d74"
DEPLOYMENT_NAME = "b22-gpt4o" 

def chatgpt_response(prompt, history):
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Prepare messages including history
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that helps people find information."
        }
    ]

    # Add previous history to the messages
    for user_prompt, bot_reply in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": bot_reply})
    
    # Add current user prompt
    messages.append({"role": "user", "content": prompt})
    
    payload = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }
  
    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}",
        headers=headers,
        json=payload
    )
  
    result = response.json()
    print(result)  # API 응답을 출력하여 확인

    if 'choices' not in result:
        return "Error: API response does not contain 'choices' key", history

    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))
    return '', history

# UI 구성
with gr.Blocks() as demo:
    gr.Markdown("#  ChatGPT World!!!")
    chatbot = gr.Chatbot()
    
    with gr.Column():
        with gr.Row():
            prompt = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요", scale=3)
            send_button = gr.Button(">", scale=1)
            clear_button = gr.Button("X")
        
        def clear_chat():
            return "", []
        
        send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
        prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
        clear_button.click(fn=clear_chat, inputs=[], outputs=[chatbot, prompt])
    
    demo.launch()


Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


In [33]:
#ui꾸미기1
import requests
import gradio as gr

AZURE_OPENAI_ENDPOINT = "https://b22-openai.openai.azure.com/openai/deployments/b22-gpt4o/chat/completions?api-version=2024-02-15-preview"
AZURE_OPENAI_API_KEY = "1dbcc70ac05e48b68207e09ed5667d74"
DEPLOYMENT_NAME = "b22-gpt4o" 

def chatgpt_response(prompt, history):
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Prepare messages including history
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that helps people find information."
        }
    ]

    # Add previous history to the messages
    for user_prompt, bot_reply in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": bot_reply})
    
    # Add current user prompt
    messages.append({"role": "user", "content": prompt})
    
    payload = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }
  
    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}",
        headers=headers,
        json=payload
    )
  
    result = response.json()
    print(result)  # API 응답을 출력하여 확인

    if 'choices' not in result:
        return "Error: API response does not contain 'choices' key", history

    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))
    return '', history

def clear_chat():
    return "", []

# UI 구성
with gr.Blocks() as demo:
    gr.Markdown("# ChatGPT World!!!\n음식점 전문가와의 대화 시작하기")
    
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 자주 묻는 질문")
            initial_questions = [
                "오늘의 추천 음식은 무엇인가요?",
                "최고의 음식점은 어디인가요?",
                "특별한 기념일에 적합한 레스토랑은?",
                "가족과 함께 가기 좋은 식당은 어디인가요?"
            ]
            for question in initial_questions:
                gr.Button(question, elem_id=f"button-{question}").click(fn=lambda q=question: q, outputs=[prompt])
    
    with gr.Row():
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(label="Chat History")
        
        with gr.Column(scale=2):
            prompt = gr.Textbox(
                label="프롬프트", 
                placeholder="프롬프트를 입력하세요", 
                lines=1,
                elem_id="input-textbox"
            )
            with gr.Row():
                send_button = gr.Button("Send", elem_id="send-button")
                clear_button = gr.Button("Clear", elem_id="clear-button")
            
            send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
            prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
            clear_button.click(fn=clear_chat, inputs=[], outputs=[chatbot, prompt])

    demo.launch()


Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1907, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1728, in postprocess_data
    if block._id in state:
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\state_holder.py", line 104, in __contains__
    block = self.blocks_config.blocks[key]
KeyError: 305


대화가통하니? []
{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '네, 물론입니다! 무엇을 도와드릴까요?', 'role': 'assistant'}}], 'created': 1722580620, 'id': 'chatcmpl-9rgCKsOIZBAUM6DD0SM9kbUWQpkLx', 'model': 'gpt-4o-2024-05-13', 'object': 'chat.completion', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'system_finge

In [48]:
import requests
import gradio as gr

AZURE_OPENAI_ENDPOINT = "https://b22-openai.openai.azure.com/openai/deployments/b22-gpt4o/chat/completions?api-version=2024-02-15-preview"
AZURE_OPENAI_API_KEY = "1dbcc70ac05e48b68207e09ed5667d74"

def chatgpt_response(prompt, history):
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Prepare messages including history
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that helps people find information."
        }
    ]

    # Add previous history to the messages
    for user_prompt, bot_reply in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": bot_reply})
    
    # Add current user prompt
    messages.append({"role": "user", "content": prompt})
    
    payload = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }
  
    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}",
        headers=headers,
        json=payload
    )
  
    result = response.json()
    print(result)  # API 응답을 출력하여 확인

    if 'choices' not in result:
        return "Error: API response does not contain 'choices' key", history

    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))
    return '', history

def clear_chat():
    return "", []

def display_question(question):
    # Return the question as a prompt but do not update history
    return question, []

# UI 구성
with gr.Blocks() as demo:
    gr.Markdown("# ChatGPT World!!!\n음식점 전문가와의 대화 시작하기")
    
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 자주 묻는 질문")
            initial_questions = [
                "오늘의 추천 음식은 무엇인가요?",
                "최고의 음식점은 어디인가요?",
                "특별한 기념일에 적합한 레스토랑은?",
                "가족과 함께 가기 좋은 식당은 어디인가요?"
            ]
            for question in initial_questions:
                gr.Button(question, elem_id=f"button-{question}").click(fn=lambda q=question: display_question(q), outputs=[prompt])
    
    with gr.Row():
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(label="Chat History")
        
        with gr.Column(scale=2):
            prompt = gr.Textbox(
                label="프롬프트", 
                placeholder="프롬프트를 입력하세요", 
                lines=1
            )
            with gr.Row():
                send_button = gr.Button("Send")
                clear_button = gr.Button("Clear")
            
            send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
            prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
            clear_button.click(fn=clear_chat, inputs=[], outputs=[chatbot, prompt])

    demo.launch()


Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    session_hash = getattr(body, "session_hash", None)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1907, in process_api
    await self.postprocess_data(block_fn, list(o), state)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1728, in postprocess_data
    prediction_value = utils.delete_none(
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\state_holder.py", line 104, in __contains__
    block = self.blocks_config.blocks[key]
KeyError: 544
Traceback (most recent call last):
  File "c:\Users\USER\A

In [51]:
import requests
import gradio as gr

AZURE_OPENAI_ENDPOINT = "https://b22-openai.openai.azure.com/openai/deployments/b22-gpt4o/chat/completions?api-version=2024-02-15-preview"
AZURE_OPENAI_API_KEY = "1dbcc70ac05e48b68207e09ed5667d74"

def chatgpt_response(prompt, history):
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Prepare messages including history
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that helps people find information."
        }
    ]

    # Add previous history to the messages
    for user_prompt, bot_reply in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": bot_reply})
    
    # Add current user prompt
    messages.append({"role": "user", "content": prompt})
    
    payload = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }
  
    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}",
        headers=headers,
        json=payload
    )
  
    result = response.json()
    print(result)  # API 응답을 출력하여 확인

    if 'choices' not in result:
        return "Error: API response does not contain 'choices' key", history

    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))
    return '', history

def clear_chat():
    return "", []

def display_question(question):
    # Return the question as a prompt but do not update history
    return question, []

# UI 구성
with gr.Blocks() as demo:
    gr.Markdown("# ChatGPT World!!!\n음식점 전문가와의 대화 시작하기")
    
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 자주 묻는 질문")
            initial_questions = [
                "오늘의 추천 음식은 무엇인가요?",
                "최고의 음식점은 어디인가요?",
                "특별한 기념일에 적합한 레스토랑은?",
                "가족과 함께 가기 좋은 식당은 어디인가요?"
            ]
            with gr.Column():
                for question in initial_questions:
                    gr.Button(question).click(
                        fn=lambda q=question: display_question(q), outputs=[prompt]
                    )
    
    with gr.Row():
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(label="Chat History")
        
        with gr.Column(scale=2):
            prompt = gr.Textbox(
                label="프롬프트", 
                placeholder="프롬프트를 입력하세요", 
                lines=1
            )
            with gr.Row():
                send_button = gr.Button("Send")
                clear_button = gr.Button("Clear")
            
            send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
            prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot])
            clear_button.click(fn=clear_chat, inputs=[], outputs=[chatbot, prompt])

    demo.launch()


Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    session_hash = getattr(body, "session_hash", None)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1907, in process_api
    await self.postprocess_data(block_fn, list(o), state)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1728, in postprocess_data
    prediction_value = utils.delete_none(
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\state_holder.py", line 104, in __contains__
    block = self.blocks_config.blocks[key]
KeyError: 593
